## Analysis of results on EVChargingEnv

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ../..

In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_style('darkgrid')

In [ ]:
period = 'Summer 2021'
logs_dir = 'logs/evcharging'

In [ ]:
# 'mpc_3' turns out to be the best performing version out of
#     'mpc_1', 'mpc_3', 'mpc_6', 'mpc_12', 'mpc_24'

dfs = []
bl_names = {
    'Offline\noptimal': 'offline_optimal',
    'Greedy': 'greedy',
    'MPC-3': 'mpc_3',
    'Random': 'random'
}
for bl_name, bl in bl_names.items():
    csv_path = os.path.join(logs_dir, bl, f'caltech_{period}.csv')
    df = pd.read_csv(csv_path)
    df['algorithm'] = bl_name
    df['trained on'] = period
    dfs.append(df)

def read_rllib_csv(name, run_name, period):
    csv_path = os.path.join(logs_dir, 'rllib', run_name, 'test_results.csv')
    df = pd.read_csv(csv_path)
    df.drop('Unnamed: 0', axis=1, inplace=True)
    df['algorithm'] = name
    df['trained on'] = period

    if 'reward_breakdown' in df.columns:
        keys = ['profit', 'carbon_cost', 'excess_charge']
        df['reward_breakdown'] = df['reward_breakdown'].map(eval).map(lambda x: tuple(x[key] for key in keys))
        df[keys] = pd.DataFrame(df['reward_breakdown'].tolist(), columns=keys)
    return df

rllib_runs = {
    # 'ppo_5e-05': 'caltech_ppo_Summer 2021_lr5e-05_seed123',
    # 'ppo_0.0005': 'caltech_ppo_Summer 2021_lr0.0005_seed123',
    'PPO': 'caltech_ppo_Summer 2021_lr5e-06_seed123',  # this one does the best

    # 'PPO-d-4': 'caltech_ppo-discrete_Summer 2021_lr0.0005_seed123',
    # 'PPO-d-5': 'caltech_ppo-discrete_Summer 2021_lr5e-05_seed123',
    'PPO-d': 'caltech_ppo-discrete_Summer 2021_lr5e-06_seed123',  # best

    # 'SAC_5e-5': 'caltech_sac_Summer 2021_lr5e-05_seed123',
    'SAC': 'caltech_sac_Summer 2021_lr5e-06_seed123',  # best
    # 'SAC-2': 'caltech_sac_Summer 2021_lr0.01_seed123',
    # 'SAC-3': 'caltech_sac_Summer 2021_lr0.001_seed123',

    'MA-PPO': 'caltech_ma0-ppo_Summer 2021_lr0.0005_seed123',  # best
    # 'MA-PPO5': 'caltech_ma0-ppo_Summer 2021_lr5e-05_seed123',
    # 'MA-PPO6': 'caltech_ma0-ppo_Summer 2021_lr5e-06_seed123',

    # 'MA-PPO-d-4': 'caltech_ma0-ppo-discrete_Summer 2021_lr0.0005_seed123',
    # 'MA-PPO-d-5': 'caltech_ma0-ppo-discrete_Summer 2021_lr5e-05_seed123',
    'MA-PPO-d': 'caltech_ma0-ppo-discrete_Summer 2021_lr5e-06_seed123',  # best

    'MA-SAC': 'caltech_ma0-sac_Summer 2021_lr0.01_seed123',  # best
    # 'MA-SAC2': 'caltech_ma0-sac_Summer 2021_lr0.001_seed123',
    # 'MA-SAC3': 'caltech_ma0-sac_Summer 2021_lr0.0001_seed123',
}
for name, run_name in rllib_runs.items():
    df = read_rllib_csv(name, run_name, period='Summer 2021')
    dfs.append(df)

rllib_ood_runs = {
    # ('PPO_5e-4', 'Summer 2019'): 'caltech_ppo_Summer 2019_lr0.0005_seed123',
    # ('PPO_5e-5', 'Summer 2019'): 'caltech_ppo_Summer 2019_lr5e-05_seed123',
    ('PPO', 'Summer 2019'): 'caltech_ppo_Summer 2019_lr5e-06_seed123',  # best

    # ('PPO-d_5e-4', 'Summer 2019'): 'caltech_ppo-discrete_Summer 2019_lr0.0005_seed123',
    ('PPO-d', 'Summer 2019'): 'caltech_ppo-discrete_Summer 2019_lr5e-05_seed123',  # best
    # ('PPO-d_5e-6', 'Summer 2019'): 'caltech_ppo-discrete_Summer 2019_lr5e-06_seed123',

    ('SAC', 'Summer 2019'): 'caltech_sac_Summer 2019_lr0.01_seed123',  # best
    # ('SAC_1e-3', 'Summer 2019'): 'caltech_sac_Summer 2019_lr0.001_seed123',
    # ('SAC_1e-4', 'Summer 2019'): 'caltech_sac_Summer 2019_lr0.0001_seed123',

    # TODO: include when done
    # ('MA-PPO', 'Summer 2019'): 'caltech_ma0-ppo_Summer 2019_lr5e-06_seed123',
    # ('MA-PPO-d', 'Summer 2019'): 'caltech_ma0-ppo-discrete_Summer 2019_lr5e-06_seed123',
    ('MA-SAC', 'Summer 2019'): 'caltech_ma0-sac_Summer 2019_lr0.01_seed123',
}
for (name, ood_period), run_name in rllib_ood_runs.items():
    df = read_rllib_csv(name, run_name, period=ood_period)
    dfs.append(df)

df = pd.concat(dfs, axis=0).reset_index(drop=True)

In [ ]:
df.groupby(['algorithm', 'trained on']).agg('mean')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 3), tight_layout=True)
ax.set(xlabel='Algorithm', ylabel='Daily Return ($)')
sns.violinplot(df, x='algorithm', y='return', hue='trained on', ax=ax,
               hue_order=['Summer 2019', 'Summer 2021'])
ax.axvline(3.5, linestyle=':', color='black')
fig.savefig('plots/evcharging/violinplot.png', bbox_inches='tight', dpi=300)